## Parallel simulations using mpi4py

In [1]:
import os,sys
Nthread = 1
os.environ["OMP_NUM_THREADS"] = str(Nthread) # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = str(Nthread) # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = str(Nthread) # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = str(Nthread) # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = str(Nthread) # export NUMEXPR_NUM_THREADS=1


In [2]:

from palik_silicon import silicon
from Photodetector import *

import autograd.numpy as np
from autograd import grad

import nlopt
import numpy as npf
from mpi4py import MPI

# import use_autograd
# use_autograd.use = 1
import rcwa
import materials, cons
from mpi_nlopt import nlopt_opt,b_filter,f_symmetry
from fft_funs import get_conv
import pickle
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()


def fun_mpi(dof,fun,N):
    '''mpi parallization for fun(dof,ctrl), ctrl is the numbering of ctrl's frequency calculation
    N calculations in total
    returns the sum: sum_{ctrl=1 toN} fun(dof,ctrl)
    '''
    dof = comm.bcast(dof)

    Nloop = int(np.ceil(1.0*N/size)) # number of calculations for each node
    val_i=[]
    g_i=[]
    val=[]
    g=[]

    for i in range(0,Nloop):
        ctrl = i*size+rank
        #print(ctrl)

        if(ctrl < N):
            color,wvlen, wvlen_index = wavelengths_tuple[ctrl];
            funi = lambda dof: fun(dof,wvlen,theta,phi, color, wvlen_index)
            grad_fun = grad(funi)

            val = funi(dof)
            gval = grad_fun(dof)

            # include indexing for now, in case one is interested
            val_i.append([ctrl,val])
            g_i.append([ctrl,gval])

    # gather the solution
    val_i = comm.gather(val_i)
    g_i = comm.gather(g_i)

    # summation
    if rank == 0:
        val_i = [x for x in val_i if x]
        g_i = [x for x in g_i if x]
        val_i = npf.concatenate(npf.array(val_i))
        g_i = npf.concatenate(npf.array(g_i))
        # sindex = val_i[:,0].argsort()

        val = np.sum(val_i[:,1])
        g = np.sum(g_i[:,1])


    val = comm.bcast(val)
    g = comm.bcast(g)
    return val,g




ModuleNotFoundError: No module named 'palik_silicon'